## 1

import library

In [36]:
import pyomo.environ as pyo

In [37]:
from pyomo.opt import SolverFactory

In [38]:
import pandas as pd

make model

In [39]:
model = pyo.AbstractModel()

creat index

In [40]:
model.I= pyo.Set()

In [41]:
model.T = pyo.Set()

In [42]:
model.G = pyo.Set()

creat parametr

In [43]:
model.price = pyo.Param(model.I, model.T, within=pyo.NonNegativeReals,)

In [44]:
model.cost = pyo.Param(model.I, model.T, within=pyo.NonNegativeReals)

In [45]:
model.holdingCost = pyo.Param(model.I,within=pyo.NonNegativeReals)

In [46]:
model.loss = pyo.Param(model.I,within=pyo.NonNegativeReals)

In [47]:
model.shortageCost= pyo.Param(model.G,within=pyo.NonNegativeReals)

In [48]:
model.leadTime= pyo.Param(model.I,within=pyo.NonNegativeReals)

In [49]:
model.capacity=pyo.Param(model.I,within=pyo.NonNegativeIntegers)

In [50]:
model.liquidity=pyo.Param(model.T,within=pyo.NonNegativeReals)

In [51]:
model.demand=pyo.Param(model.I,model.T,within=pyo.NonNegativeReals)

In [52]:
model.init_inventory=pyo.Param(model.I,default=0)

In [53]:
model.supplier_of = pyo.Param(model.I)

creat variable

In [54]:
model.x = pyo.Var(model.I,model.T,within=pyo.NonNegativeIntegers)

In [55]:
model.sell = pyo.Var(model.I,model.T,within=pyo.NonNegativeIntegers)

In [56]:
model.inventory = pyo.Var(model.I,model.T,within=pyo.NonNegativeIntegers)

In [57]:
model.lost = pyo.Var(model.I,model.T,within=pyo.NonNegativeIntegers)

In [58]:
model.y = pyo.Var(model.G,model.T,within=pyo.Binary)

make objective function



In [59]:
def obj_rule(model):
    expr1 = sum(sum(model.price[i,t] * model.sell[i,t] - model.cost[i,t] * model.x[i,t] - model.holdingCost[i] * model.inventory[i,t]- model.loss[i]*model.lost[i,t]  for i in model.I) for t in model.T)
    expr2 = sum(sum(model.shortageCost[g]*model.y[g,t] for g in model.G) for t in model.T)
    return expr1 - expr2
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)


## 2

### make liqudity constraint


In [60]:
def constraint_liquidity_rule(model, t):
    return sum(model.x[i, t] * model.cost[i, t] for i in model.I) <= model.liquidity[t]

model.liquidity_constraint = pyo.Constraint(model.T, rule=constraint_liquidity_rule)


### make capacity rule

def constraint_capacity_rule(model, i, t):
    if t == min(model.T): 
        prev_inv = model.init_inventory[i]
    else:
        prev_inv = model.inventory[i, t - 1]
    x=0
    x = sum(model.x[i, τ] for τ in model.T 
               if (τ + model.leadTime[i] in model.T) and (τ + model.leadTime[i] == t))

    return prev_inv + x <= model.capacity[i]

model.Capacity_Constraint = pyo.Constraint(model.I, model.T, rule=constraint_capacity_rule)


In [61]:
from pyomo.core.expr.current import identify_variables

def constraint_capacity_rule(model, i, t):
    # موجودی قبلی
    if t == model.T.first():
        prev_inv = model.init_inventory[i]
    else:
        prev_inv = model.inventory[i, model.T.prev(t)]

    # lead time (safe cast)
    lead = int(pyo.value(model.leadTime[i]))

    # نگاشت ایندکس‌ها به لیست برای کار با مجموعه‌های رشته‌ای/عددی
    t_list = list(model.T)
    t_pos = t_list.index(t)

    # جمع سفارش‌هایی که موعدشان به t می‌رسد
    arrivals = [ model.x[i, tau] for tau in t_list
                 if t_list.index(tau) + lead == t_pos ]

    x = pyo.quicksum(arrivals)   # همیشه یک Pyomo expression می‌سازد

    # بررسی می‌کنیم آیا عبارت شامل متغیر (Var) هست یا خیر
    # اگر هیچ متغیری نداشت (عبارت ثابت شد)، به Pyomo بگوییم این قید "Feasible" است
    diff = prev_inv + x - model.capacity[i]
    vars_in_expr = list(identify_variables(diff))
    if len(vars_in_expr) == 0:
        return pyo.Constraint.Feasible

    # در غیر این صورت قید عادی را برگردان
    return prev_inv + x <= model.capacity[i]

model.Capacity_Constraint = pyo.Constraint(model.I, model.T, rule=constraint_capacity_rule)


### make balance rule

In [62]:
def constraint_balance_rule(model, i, t):
    if t == min(model.T): 
        prev_inv = model.init_inventory[i]
    else:
        prev_inv = model.inventory[i, t - 1]
    x=0
    x = sum(model.x[i, τ] for τ in model.T 
               if (τ + model.leadTime[i] in model.T) and (τ + model.leadTime[i] == t))

    return model.inventory[i,t] == prev_inv + x - model.sell[i,t]

model.balance_Constraint = pyo.Constraint(model.I, model.T, rule=constraint_balance_rule)


### make demand rule

In [63]:
def constraint_demand_rule(model, i, t):
    return model.sell[i,t] + model.lost[i,t] == model.demand[i,t]

model.demand_Constraint = pyo.Constraint(model.I, model.T, rule=constraint_demand_rule)


### make group rule

In [64]:
def supplier_activation_rule(model, i, t):
    g = model.supplier_of[i]
    return model.x[i, t] <= 1e6 * model.y[g, t]
model.SupplierLink = pyo.Constraint(model.I, model.T, rule=supplier_activation_rule)


## about dat file

In [65]:
instance = model.create_instance('data2.dat')

In [66]:
solver = SolverFactory('glpk')
results = solver.solve(instance, tee=True)


GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\shaya\AppData\Local\Temp\tmpan_h22jx.glpk.raw --wglp C:\Users\shaya\AppData\Local\Temp\tmp67dk46jj.glpk.glp
 --cpxlp C:\Users\shaya\AppData\Local\Temp\tmpwo3kk1wx.pyomo.lp
Reading problem data from 'C:\Users\shaya\AppData\Local\Temp\tmpwo3kk1wx.pyomo.lp'...
C:\Users\shaya\AppData\Local\Temp\tmpwo3kk1wx.pyomo.lp:4812: warning: lower bound of variable 'x578' redefined
C:\Users\shaya\AppData\Local\Temp\tmpwo3kk1wx.pyomo.lp:4812: warning: upper bound of variable 'x578' redefined
545 rows, 585 columns, 1420 non-zeros
585 integer variables, 9 of which are binary
4821 lines were read
Writing problem data to 'C:\Users\shaya\AppData\Local\Temp\tmp67dk46jj.glpk.glp'...
4102 lines were written
GLPK Integer Optimizer, v4.65
545 rows, 585 columns, 1420 non-zeros
585 integer variables, 9 of which are binary
Preprocessing...
137 constraint coefficient(s) were reduced
425 rows, 475 columns, 1156 non-zeros
4

In [72]:
import pyomo.environ as pyo

solver = pyo.SolverFactory('glpk')

def solve_and_get_obj(inst):
    solver.solve(inst, tee=False)
    return pyo.value(inst.obj)

# ========= FIX: تحلیل روی instance =========
instance = model.create_instance('data2.dat')

# ------------------------------
# Sensitivity: holdingCost
# ------------------------------
print("=== Sensitivity: holdingCost ===")
for hc in [0.5, 1.0, 2.0]:
    for i in instance.I:
        instance.holdingCost[i] = hc
    obj_value = solve_and_get_obj(instance)
    print(f"holdingCost = {hc} → Obj = {obj_value:.2f}")

# ------------------------------
# Sensitivity: shortageCost
# ------------------------------
print("\n=== Sensitivity: shortageCost ===")
for sc in [5, 10, 20]:
    for i in instance.I:
        instance.shortageCost[i] = sc
    obj_value = solve_and_get_obj(instance)
    print(f"shortageCost = {sc} → Obj = {obj_value:.2f}")

# ------------------------------
# Sensitivity: price
# ------------------------------
print("\n=== Sensitivity: price ===")
for factor in [0.9, 1.0, 1.1]:
    for i in instance.I:
        for t in instance.T:
            instance.price[i, t] = instance.price[i, t] * factor
    obj_value = solve_and_get_obj(instance)
    print(f"price ×{factor} → Obj = {obj_value:.2f}")


=== Sensitivity: holdingCost ===


TypeError: Attempting to set the value of the immutable parameter holdingCost[i1] after the parameter has been constructed.  If you intend to change the value of this parameter dynamically, please declare the parameter as mutable [i.e., Param(mutable=True)]